In [85]:
def load_file(filename):
  with open(filename) as f:
    raw = f.read()
    [stacks, moves] = raw.split('\n\n')
    return [stacks.split('\n'), moves.strip().split('\n')]

In [72]:
import math
import torch
import re

def parse_chunk(chunk):
  if chunk.strip() == '':
    return 0
  char = chunk[1] 
  return ord(char)

def parse_stack(stack):
  chunks = []
  for i in range(math.ceil(len(stack) / 4)):
    chunks.append(stack[i*4:(i+1)*4])
  return [parse_chunk(chunk) for chunk in chunks]
  
def parse_move(move):
  regex = r"move (\d+) from (\d+) to (\d+)"
  matches = re.search(regex, move)
  values = matches.groups()
  return [int(x) for x in values]

def parse_file(filename):
  stacks, moves = load_file(filename)
  stacks = torch.tensor([parse_stack(stack) for stack in stacks[:-1]])
  stacks = torch.flip(stacks.T, [1]).numpy().tolist()
  stacks = [list(filter(lambda x: x != 0, stack)) for stack in stacks]
  moves = [parse_move(move) for move in moves]
  return [stacks, moves]


In [73]:
parse_file("example.txt")

[[[90, 78], [77, 67, 68], [80]], [[1, 2, 1], [3, 1, 3], [2, 2, 1], [1, 1, 2]]]

In [76]:
def apply_move(X, move):
  [n, from_stack, to_stack] = move
  for _ in range(n):
    X[to_stack - 1].append(X[from_stack - 1].pop())

  return X

In [77]:
x = parse_file("example.txt")
apply_move(x[0], x[1][0])

[[90, 78, 68], [77, 67], [80]]

In [78]:
def run_moves(X, moves):
  for move in moves:
    apply_move(X, move)
  return X

In [80]:
def get_answer(f):
  stacks, moves = parse_file(f)
  cols = run_moves(stacks, moves)
  s = ""
  for c in cols:
    if len(c) == 0: continue
    s += chr(c.pop())

  return s

In [81]:
[X, moves] = parse_file("example.txt")
run_moves(X, moves)

[[67], [77], [80, 68, 78, 90]]

In [83]:
get_answer("example.txt")

'CMZ'

In [86]:
get_answer("input.txt")

'VRWBSFZWM'

In [90]:
def run_moves_2(X, moves):
  for move in moves:
    apply_move_2(X, move)
  return X

In [95]:
def apply_move_2(X, move):
  [n, from_stack, to_stack] = move
  crates = []
  for _ in range(n):
    crates.append(X[from_stack - 1].pop())
  crates.reverse()
  X[to_stack - 1] = X[to_stack - 1] + crates

  return X

In [96]:
def get_answer_2(f):
  stacks, moves = parse_file(f)
  cols = run_moves_2(stacks, moves)
  s = ""
  for c in cols:
    if len(c) == 0: continue
    s += chr(c.pop())

  return s

In [97]:
get_answer_2("example.txt")

'MCD'

In [98]:
get_answer_2("input.txt")

'RBTWJWMCF'